#Tal Birs 208574905
#Maya David 209282532

In [1]:
import pandas as pd
df= pd.read_csv("spam_ham_dataset.csv")
df = df[['text', 'label_num']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   text       5171 non-null   object
 1   label_num  5171 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 80.9+ KB


apply token in the df

In [2]:
from nltk import word_tokenize

df['tokens'] = df['text'].apply(lambda x: word_tokenize(x))

In [3]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(lowercase=False,
                             preprocessor=lambda x: x,
                             tokenizer=lambda x: x)

In [5]:
docs = df['tokens'].tolist()

In [ ]:
vectorizer.fit(docs)
vectorizer.get_feature_names()


In [ ]:
vectorizer.transform(docs)
vectorizer.fit_transform(docs)

In [8]:
df_ham = df[df['label_num'] == 0]
df_spam = df[df['label_num'] == 1]

build the matrix of the text and counter

In [9]:
def get_matrix(text_df_input, model_input):
    matrix = model.fit_transform(text_df_input).toarray()
    return pd.DataFrame(data=matrix, columns=model_input.get_feature_names())

split the data to train and test

In [15]:
from sklearn.model_selection import train_test_split

X = df.drop('label_num', axis=1)
y = df['label_num']
train, test = train_test_split(df, test_size=0.2, random_state=1234)


show the features of the text

In [ ]:
vocabulary = []
for list in df['tokens']:
    vocabulary += list
wordset = set(vocabulary)

count_vectorizer = CountVectorizer(vocabulary=wordset, lowercase=False)
print(count_vectorizer.get_feature_names_out())

X_train = count_vectorizer.fit_transform(train['text'])
df_bow_train = pd.DataFrame(X_train.toarray(), columns=count_vectorizer.get_feature_names_out())
print(df_bow_train.head())
print(f'train with {X_train.shape[0]} records with {X_train.shape[1]} features')

X_test = count_vectorizer.fit_transform(test['text'])
df_bow_test = pd.DataFrame(X_test.toarray(), columns=count_vectorizer.get_feature_names_out())
print(df_bow_test.head())
print(f'test with {X_test.shape[0]} records with {X_test.shape[1]} features')

y_train = train['label_num']
y_test = test['label_num']

build KNN classifier

In [ ]:
#KNeighborsClassifier

from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.neighbors import KNeighborsClassifier
import math

k= int(math.sqrt(X_train.shape[0])/2)
if k%2 ==0:
    k+=1

clf= KNeighborsClassifier(n_neighbors=k)
clf.fit(X_train,y_train)
y_pred_test =clf.predict(X_test)

knn_acc_score= accuracy_score(y_test, y_pred_test)
knn_pre_score=precision_score(y_test, y_pred_test,average='weighted')
knn_recall_score=recall_score(y_test, y_pred_test,average='macro')
print("test scores:")
print("Accuracy:",knn_acc_score)
print("Precision:",knn_pre_score )
print("recall:",knn_recall_score )


build SVM model

In [ ]:
#create svm model
from sklearn import svm
from sklearn import metrics


model = svm.SVC()
model.fit(X_train,y_train)

y_pred_test=model.predict(X_test)
svm_acc_score= metrics.accuracy_score(y_test,y_pred_test)
svm_pre_score=metrics.recall_score(y_test, y_pred_test)
svm_recall_score=metrics.precision_score(y_test, y_pred_test)
svm_f_score=metrics.f1_score(y_test,y_pred_test)

print("accuracy",svm_acc_score)
print("recall:",svm_recall_score)
print("precision",svm_pre_score)
print("f1:",svm_f_score)



build Logistic Regression model

In [ ]:
#LogisticRegression
from sklearn import metrics
from sklearn.linear_model import LogisticRegression

clf= LogisticRegression(random_state=1234)
clf.fit(X_train,y_train)

y_pred_test =clf.predict(X_test)
logistic_acc_score=metrics.accuracy_score(y_test, y_pred_test)
logistic_pre_score=metrics.precision_score(y_test, y_pred_test)
logistic_recall_score=metrics.recall_score(y_test, y_pred_test)
logistic_f_score= metrics.f1_score(y_test, y_pred_test)

print("for test prediction: ")
print("Accuracy:",logistic_acc_score)
print("Precision:", logistic_pre_score)
print("recall:", logistic_recall_score)
print("F1:",logistic_f_score)


build Decision Tree Classifier

In [ ]:
#DecisionTreeClassifier

from sklearn.metrics import plot_confusion_matrix
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(random_state=1234)
model.fit(X_train, y_train)
plt.figure(figsize=(15, 20), dpi=100)


y_pred_test = model.predict(X_test)
cm_test = pd.crosstab(y_test, y_pred_test, colnames=['pred'], margins=True)
plot_confusion_matrix(model, X_test, y_test)
plt.show()

tree_acc_score= metrics.accuracy_score(y_test, y_pred_test)
tree_pre_score=metrics.precision_score(y_test, y_pred_test)
tree_recall_score= metrics.recall_score(y_test, y_pred_test)
tree_f_score=metrics.f1_score(y_test, y_pred_test)

print("Accuracy:", tree_acc_score)
print("Precision:", tree_pre_score)
print("recall:",tree_recall_score)
print("F1:", tree_f_score)


show all the scores in one table

In [48]:
from tabulate import tabulate

#create data
data = [["Logistic Regression", "Accuracy:",logistic_acc_score, "Precision:",logistic_pre_score, "recall:",                 logistic_recall_score, "F1:", logistic_f_score],
        ["SVM", "Accuracy:",svm_acc_score,"precision:",svm_pre_score,"recall:",svm_recall_score,"f1",svm_f_score,],
        ["Decision Tree", "Accuracy:",tree_acc_score, "Precision:",tree_pre_score,"recall:",tree_recall_score, "F1:",tree_f_score],
        ["KNN", "Accuracy:",knn_acc_score,"Precision:",knn_pre_score,"recall:",knn_recall_score]]

#define header names
col_names = ["model_name", "scores"]


print(tabulate(data, tablefmt="simple" ))


-------------------  ---------  --------  ----------  --------  -------  --------  ---  --------
Logistic Regression  Accuracy:  0.978744  Precision:  0.955128  recall:  0.973856  F1:  0.964401
SVM                  Accuracy:  0.960386  precision:  0.964052  recall:  0.907692  f1   0.935024
Decision Tree        Accuracy:  0.943961  Precision:  0.907895  recall:  0.901961  F1:  0.904918
KNN                  Accuracy:  0.822222  Precision:  0.85834   recall:  0.844408
-------------------  ---------  --------  ----------  --------  -------  --------  ---  --------


explain about the scores

In [ ]:
#the best scores are in logistic regression model and in SVM model
# Both models is used to solve binary problems and is therefore suitable for the problem presented #in this data frame.

In [ ]:
#SVMs are used for classification and regression problems, but mostly classification. The algorithm creates a hyperplane or line to classify data. It finds the best line separator using the kernel trick (decision boundary that has same distance from the boundary point of both classes). It's a powerful way to learn complex nonlinear functions.
#Logistic regression helps classify data. It describes data and explains variable relationships. Logistic regression is used when the output variable (y) ranges between 1 (yes) and 0 (no).
